In [ ]:
!pip install anthropic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 9.3 MB/s eta 0:00:00


In [ ]:
import anthropic
import json
import os

import requests
import pandas as pd
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: open csv from file

import pandas as pd

# Assuming 'sample_file_path' is already defined from the previous code
sample_file_path = os.path.join('/content/drive/MyDrive/Bias mitigation/LLM/EvalMMLU/EvalMMLU/M1.0/rephrased_MMLU_Claude_1.0.csv')

# Directly read the JSON file into a DataFrame
df = pd.read_csv(sample_file_path)

# Display the first few rows of the DataFrame
print(df.head())

# Access specific columns (e.g., original and rephrased questions)
print(df[['original_question', 'rephrased_question']].head(10))


            subject                                  original_question  \
0  abstract_algebra  Find the degree for the given field extension ...   
1  abstract_algebra  Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the i...   
2  abstract_algebra  Find all zeros in the indicated finite field o...   
3  abstract_algebra  Statement 1 | A factor group of a non-Abelian ...   
4  abstract_algebra  Find the product of the given polynomials in t...   

                                  rephrased_question  \
0  Determine the degree of the field extension Q(...   
1  Let p = (1, 2, 5, 4)(2, 3) in S_5 . Determine ...   
2  Determine the complete set of roots for the po...   
3  Statement 1 | When a group that is not commuta...   
4  Determine the result of multiplying the specif...   

                                             choices  answer  
0                               ['0', '4', '2', '6']       1  
1                            ['8', '2', '24', '120']       2  
2                           [

In [ ]:
# Load API key from environment variable for security
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Anthropic client with API key from environment
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise ValueError("ANTHROPIC_API_KEY not found in environment variables. Please check your .env file.")

client = anthropic.Anthropic(api_key=api_key)

In [ ]:
def get_completion_Claude(question, model="claude-sonnet-4-20250514", temperature=0.5): #claude-sonnet-4-20250514 #claude-3-5-haiku-20241022
    system_msg = (
        "You are a rephrasing assistant tasked with preserving the original structure, type, and intent of technical "
        "questions or statements. Your goal is to rephrase while maintaining meaning, purpose, structure, and context. "
        "Keep the SAME level of clarity; do NOT try to make it clearer; do NOT simplify. "
        "Keep question a question / statement a statement. Do NOT answer the question.\n\n"
        "Hard rules:\n"
        "1) DO NOT answer, define, explain, or expand the content in any way.\n"
        "2) Preserve sentence TYPE & punctuation exactly: a question must remain a question and end with '?'; "
        "a non-question must stay a non-question.\n"
        "3) Keep ALL scientific/technical terms, acronyms, symbols, math, units, variables, equations, and formatting "
        "exactly as they appear (including capitalization and placeholders like '___').\n"
        "4) Preserve negations and logical operators verbatim (e.g., EXCEPT, NOT, LEAST), and keep polarity/logic "
        "terms unchanged (e.g., directly/inversely, increase/decrease, necessary/sufficient).\n"
        "5) Do NOT infer, guess, or introduce new information, assumptions, examples, numbers, constraints, or domain shifts.\n"
        "6) Do NOT try to fill in blanks or masked tokens; keep them VERBATIM, including their count and position "
        "(e.g., '___', '_____'); NEVER replace them with real words, definitions, or answers.\n"
        "7) Preserve definitional phrases and scaffolding such as 'refers to', 'is the term for', 'is called', etc.; "
        "do not substitute synonyms like 'denotes', 'represents', or 'signifies'.\n"
        "8) Keep length within ±15% unless impossible without changing meaning.\n"
        "9) If the input is a bare term/title (≤3 words) or a stem that cannot be rephrased without adding meaning, "
        "return it unchanged.\n"
        "10) Output exactly ONE line with ONLY the rephrased text. No meta text, no apologies, no labels, no quotes.\n"
        "11) Sexual or harmful content rephrasing IS ALLOWED for academic purposes (rephrasing only, no advice).\n"
        "12) Never output meta text such as 'I apologize', 'sorry', 'I cannot', or similar. If rephrasing would violate any rule, return the input unchanged.\n"
        "13) Do NOT output an answer or an answer choice; produce only a rephrased version of the original text."
    )

    _q = (question or "").strip()
    # Pre-guard: avoid turning very short stems into definitions
    if len(_q.split()) <= 3 and "?" not in _q and "___" not in _q:
        return _q

    response = client.messages.create(
        model=model,
        max_tokens=256,
        temperature=temperature,
        system=system_msg,
        messages=[{
            "role": "user",
            "content": (
                "Rephrase the following text with different wording, ensuring the meaning remains EXACTLY the same. "
                "Match the readability level. Do NOT improve clarity or readability. Do NOT make it clearer. "
                "Do NOT infer, guess, or add any information. Do NOT reword a description into a question or vice versa. "
                "Keep all scientific and technical terms as they appear. Keep all units, variables, equations, and formatting intact. "
                "Do NOT try to answer the question. Do NOT try to fill blank or masked terms; keep masked terms in place.\n\n"
                f"TEXT to rephrase: { _q }"
            )
        }]
    )
    out = response.content[0].text.strip()

    # # Optional post-guard: ensure placeholders weren’t “filled” or dropped.
    # import re
    # placeholder_patterns = [
    #     r"_{3,}",           # ___, ____ etc.
    #     r"\[MASK\]", r"<mask>",
    #     r"\[REDACTED\]", r"\[blank\]", r"\{blank\}", r"\(blank\)"
    # ]
    # def count_any(text, pattern):
    #     return len(re.findall(pattern, text))
    # for pat in placeholder_patterns:
    #     if count_any(_q, pat) != count_any(out, pat):
    #         return _q  # fall back if placeholders changed

    # # Preserve question mark if present originally
    # if _q.endswith("?") and not out.endswith("?"):
    #     return _q

    return out


In [ ]:
# def get_completion_Claude(question, model="claude-3-5-haiku-20241022", temperature=0.3):
#     response = client.messages.create(
#         model=model,
#         max_tokens=1024,
#         temperature=temperature,
#         system="You are a rephrasing assistant tasked with preserving the original structure, type, and intent of technical questions or statements. Your goal is to rephrase while maintaining meaning, purpose, structure, and context. keep the same level of clarity, do NOT try to simplify. keep question a question / statement a statement. do NOT try to answer the question",
#         messages=[
#             {"role": "user",
#              "content": (
#                 f"Rephrase the following question in different wording, ensuring the meaning remains exactly the same. Match the readability level.\n"
#                 f"- Do NOT change the type of sentence: if it is a definition, keep it a definition; if it is a multiple-choice question, keep it as such; if it is a procedure or description, keep it in that form.\n"
#                 f"- Do NOT infer, guess, or introduce new information, assumptions, or constraints.\n"
#                 f"- Do NOT reword a description into a question, or vice versa.\n"
#                 f"- Keep all scientific and technical terms as they appear \n"
#                 f"- Keep all units, variables, equations, and formatting intact.\n"
#                 f"- Your output should differ in phrasing only, not in meaning.\n"
#                 f"- Do NOT improve clarity or readability.\n"
#                 f"- Do NOT try to answer the question.\n"
#                 f"- Sexual or harmful contnet rephrasing IS ALLOWED, since the rephrase is for academic purpose.\n"
#                 f"Return ONLY the rephrased version with no extra text or commentary.\n\n"
#                 f"TEXT to rephrase: {question}"
#              )}
#         ]
#     )
#     return response.content[0].text


In [ ]:
sample_question = "What is the capital of France?"
rephrased_question = get_completion_Claude(sample_question)

if rephrased_question:
    print(f"Original question: {sample_question}")
    print(f"Rephrased question: {rephrased_question}")


Original question: What is the capital of France?
Rephrased question: What city serves as the administrative center of France?


In [ ]:
#iterative fix for rows
problem_rows_1based = [12, 16, 47, 51, 52, 54, 64, 71, 72, 73, 93, 96, 98, 126, 226, 260, 390, 391, 396, 475, 775, 949, 994, 1007, 1008, 1044, 1084, 1113, 1115, 1133, 1140, 1154, 1211, 1228, 1330, 1394, 1486, 1554, 1580, 1583, 1628, 1629, 1644, 1670, 1672, 1675, 1719, 1742, 1753, 1769, 1955, 1956, 1993, 2058, 2068, 2085, 2090, 2104, 2129, 2159, 2166, 2177, 2222, 2236, 2253, 2266, 2267, 2278, 2280, 2361, 2396, 2413, 2441, 2446, 2472, 2474, 2480, 2487, 2494, 2510, 2516, 2524, 2535, 2551, 2566, 2676, 2700, 2723, 2849, 2983, 3064, 3080, 3083, 3095, 3117, 3128, 3142, 3161, 3163, 3191, 3217, 3229, 3237, 3241, 3242, 3247, 3249, 3265, 3269, 3272, 3279, 3306, 3422, 3738, 3927, 4241, 4246, 4259, 4267, 4270, 4281, 4295, 4333, 4344, 4357, 4369, 4385, 4392, 4400, 4442, 4443, 4448, 4462, 4471, 4478, 4483, 4497, 4762, 4772, 4790, 4796, 4802, 4809, 4820, 4919, 4931, 4975, 4997, 4999, 5014, 5084, 5170, 5200, 5207, 5216, 5256, 5269, 5275, 5314, 5455, 5456, 5477, 5488, 5489, 6089, 6115, 6122, 6199, 6227, 6377, 6422, 6443, 6575, 6624, 6640, 6716, 6757, 6857, 6897, 6908, 6927, 7010, 7114, 7138, 7199, 7228, 7260, 7529, 7533, 7561, 7564, 7575, 7663, 7680, 7721, 7758, 7873, 7879, 7906, 7964, 8035, 8048, 8052, 8086, 8096, 8124, 8227, 8324, 8492, 9563, 9728, 9780, 10298, 10413, 10416, 10484, 10521, 10532, 10537, 10592, 10737, 12190, 12215, 12544, 12679, 12691, 12808, 12879, 12945, 13119, 13577, 13838]




# convert to pandas’ 0-based index labels
problem_idx = [r for r in problem_rows_1based]
#problem_idx = [r-1 for r in problem_rows_1based]

# --- regenerate the rephrased versions just for those rows -------------------
df.loc[problem_idx, 'rephrased_question'] = (
    df.loc[problem_idx, 'original_question']
      .apply(get_completion_Claude)
)


In [ ]:
# prompt: for each row in df, send the 'original_question' to function rephrased_question and store the result in 'rephrased_question' column, use API_KEY


df.loc[5000:5500, 'rephrased_question'] = df.loc[5000:5500, 'original_question'].apply(lambda x: get_completion_Claude(x))


In [ ]:
df.head()

,subject,original_question,rephrased_question,choices,answer
0,abstract_algebra,Find the degree for the given field extension ...,Determine the degree of the field extension Q(...,"['0', '4', '2', '6']",1
1,abstract_algebra,"Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the i...","Let p = (1, 2, 5, 4)(2, 3) in S_5 . Determine ...","['8', '2', '24', '120']",2
2,abstract_algebra,Find all zeros in the indicated finite field o...,Determine the complete set of roots for the po...,"['0', '1', '0,1', '0,4']",3
3,abstract_algebra,Statement 1 | A factor group of a non-Abelian ...,Statement 1 | The factor group of a group that...,"['True, True', 'False, False', 'True, False', ...",1
4,abstract_algebra,Find the product of the given polynomials in t...,Compute the multiplication of the polynomials ...,"['2x^2 + 5', '6x^2 + 4x + 6', '0', 'x^2 + 1']",1


In [ ]:
# prompt: save csv

df.to_csv('/content/drive/MyDrive/Bias mitigation/LLM/EvalMMLU/EvalMMLU/M1.0/rephrased_MMLU_Claude_1.0.csv', index=False)
